In [6]:
import sys
import os
import shutil
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
#tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
#from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.1.5
Tensorflow version: 1.14.0


In [7]:
# top k items to recommend
TOP_K = 10


# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [8]:
df = pd.read_csv('ratings_Electronics.csv') # load the dataset
df=pd.DataFrame(df)
df.columns = ["userID","itemID","rating","timestamp"] #adding column head, convert rating into purchase

In [9]:
df

,userID,itemID,rating,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [10]:
#split the dataset for training
from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.datasets.download_utils import maybe_download
from recommenders.datasets.python_splitters import python_random_split,python_chrono_split,python_stratified_split

from recommenders.datasets.spark_splitters import spark_random_split
#df_train, df_test = python_random_split(df, ratio=0.7)

In [11]:
train, test = python_chrono_split(df, ratio=0.75, filter_by="user",col_user='userID', col_item='itemID', col_timestamp='timestamp')

In [12]:
# functions for ncf dataset#
data=NCFDataset(train=train, test=test, seed=SEED)

MemoryError: 

In [ ]:
#train_a=train
#train_a['userID']=train_a['user_id']
test['rating']=test['rating']
test['timestamp']=test['timestamp']

In [10]:
#test.drop(test.columns[[0]], axis=1, inplace=True)# drop fourth column

In [11]:
#test.columns = test.columns.str.replace(", ",",")

In [14]:
#test.head()